## small gift for birthday

dear friends!

recently bostrom had his first birthday!

since the main gift cannot be unlocked until we reach the goal, the cybercongress decided to make a small gift

soon each citizen who claim the gift will receive 1A and 1V

this will enable your contribution to the development of a bootloader for superintelligence right now

we thank everyone for participating

In [ ]:
import pandas as pd
import json
import requests
from tqdm import tqdm
import base64
import ast

from cyber_sdk.core.bank import MsgMultiSend, MultiSendInput, MultiSendOutput
from cyber_sdk.core import Coins
from cyber_sdk.client.lcd.api.tx import CreateTxOptions, SignerOptions
from cyber_sdk.core.fee import Fee
from cyber_sdk.client.lcd import LCDClient
from cyber_sdk.core.bech32 import AccAddress

GRAPHQL_API = 'https://titan.cybernode.ai/graphql/v1/graphql'
HEADERS = {
    'content-type': 'application/json',
    "authorization": ''
}
GET_TXS = False

### Get Transactions from GraphQL API

In [2]:
def run_graphql_query(query: str) -> list:
    # A simple function to use requests.post to make the API call. Note the json= section.
    request = requests.post(GRAPHQL_API, json={'query': query}, headers=HEADERS)
    if request.status_code == 200:
        return request.json()['data']['message']
    else:
        raise Exception(f'Query failed to run by returning code of {request.status_code}\nquery: {query}')


def parse_res(res: list) -> pd.DataFrame:
    temp_list = []
    for res_item in tqdm(res):
        msg = ast.literal_eval(base64.b64decode(res_item['value']['msg']).decode("utf-8"))
        sender = res_item['value']['sender']
        contract = res_item['value']['contract']
        msg_type = res_item['type']
        transaction_hash = res_item['transaction_hash']
        height = res_item['transaction']['block']['height']
        temp_list.append([height, transaction_hash, msg_type, sender, contract, msg])
    return pd.DataFrame(temp_list, columns=['height', 'transaction_hash', 'type', 'sender', 'contract', 'msg'])

In [3]:
graphql_query = '''{
  message(where: {type: {_eq: "cosmwasm.wasm.v1.MsgExecuteContract"}}) {
    transaction_hash
    type
    value
    transaction {
      block {
        height
      }
    }
  }
}'''

if GET_TXS:
    print('Getting data from GraphQL API')
    graphql_res = run_graphql_query(query=graphql_query)
    print('Parsing data')
    contract_txs_df = parse_res(res=graphql_res)
    contract_txs_df.to_csv('data/claim_parser.csv')

### Get Claims

In [4]:
contract_txs_df = pd.read_csv('data/claim_parser.csv')
contract_txs_df.head()

,Unnamed: 0,height,transaction_hash,type,sender,contract,msg
0,0,4278876,576816E54BE36F2C121DFDA06925FAAF6D4E0B1EF574E8...,cosmwasm.wasm.v1.MsgExecuteContract,bostrom1fd8kd7qkneye4y96slq8992r34lt0wfexd0e3t,bostrom1xut80d09q0tgtch8p0z4k5f88d3uvt8cvtzm5h...,{'create_passport': {'avatar': 'Qmcvsz4VezuUm8...
1,1,4248054,55C21ED7760BACD35AFA24E72D81D12F1A1EF3565A1855...,cosmwasm.wasm.v1.MsgExecuteContract,bostrom1usrgyu7aul0pns4nltdwl2saxg67ckyl4npf0f,bostrom1xut80d09q0tgtch8p0z4k5f88d3uvt8cvtzm5h...,{'proof_address': {'address': 'osmo1sh992rngek...
2,2,4248063,A6B3AEA82FA5176BD1EE818E3BAA8F95CB10D9F0AB6974...,cosmwasm.wasm.v1.MsgExecuteContract,bostrom1usrgyu7aul0pns4nltdwl2saxg67ckyl4npf0f,bostrom16t6tucgcqdmegye6c9ltlkr237z8yfndmasrhv...,{'claim': {'proof': ['5c26776d9f7a6a02a76998d2...
3,3,4248063,A6B3AEA82FA5176BD1EE818E3BAA8F95CB10D9F0AB6974...,cosmwasm.wasm.v1.MsgExecuteContract,bostrom1usrgyu7aul0pns4nltdwl2saxg67ckyl4npf0f,bostrom16t6tucgcqdmegye6c9ltlkr237z8yfndmasrhv...,{'claim': {'proof': ['622694c857234e2161ed3c5e...
4,4,4250355,ECA07FA8F573B9FB8BAD1EE4257E8AAEDA514FD2A6A927...,cosmwasm.wasm.v1.MsgExecuteContract,bostrom1734emz8rm2r54tf5gqj226dvv4phx95fnr5e9l,bostrom1xut80d09q0tgtch8p0z4k5f88d3uvt8cvtzm5h...,{'create_passport': {'avatar': 'QmW4NYSoQHn4Rj...


In [5]:
claim_txs_df =
contract_txs_df[contract_txs_df.contract == 'bostrom16t6tucgcqdmegye6c9ltlkr237z8yfndmasrhvh7ucrfuqaev6xq7cpvek'][
    ['sender', 'msg']]
claim_txs_df.loc[:, 'msg_json'] = claim_txs_df.msg.map(lambda x: json.loads(x.replace("'", '"')))
claim_txs_df.loc[:, 'method_name'] = claim_txs_df.msg_json.map(lambda x: list(x.keys())[0])
claim_txs_df = claim_txs_df[claim_txs_df.method_name == 'claim']
claim_txs_df.loc[:, 'nickname'] = claim_txs_df.msg_json.map(lambda x: x['claim']['nickname'])
claim_txs_df.loc[:, 'gift_claiming_address'] = claim_txs_df.msg_json.map(lambda x: x['claim']['gift_claiming_address'])
claim_txs_df.loc[:, 'gift_amount'] = claim_txs_df.msg_json.map(lambda x: int(x['claim']['gift_amount']))

print(f'Number of claims: {len(claim_txs_df):>,}')
print(f'Number of claiming addresses: {claim_txs_df.gift_claiming_address.nunique():>,}')
print(f'Number of recipient: {claim_txs_df.sender.nunique():>,}')
claim_txs_df.head()

Number of claims: 8,815
Number of claiming addresses: 8,773
Number of recipient: 5,777


,sender,msg,msg_json,method_name,nickname,gift_claiming_address,gift_amount
2,bostrom1usrgyu7aul0pns4nltdwl2saxg67ckyl4npf0f,{'claim': {'proof': ['5c26776d9f7a6a02a76998d2...,{'claim': {'proof': ['5c26776d9f7a6a02a76998d2...,claim,jimbosuson,osmo1usrgyu7aul0pns4nltdwl2saxg67ckyl7mx28u,91000000
3,bostrom1usrgyu7aul0pns4nltdwl2saxg67ckyl4npf0f,{'claim': {'proof': ['622694c857234e2161ed3c5e...,{'claim': {'proof': ['622694c857234e2161ed3c5e...,claim,jimbosuson,cosmos1usrgyu7aul0pns4nltdwl2saxg67ckylkq463w,91000000
6,bostrom1734emz8rm2r54tf5gqj226dvv4phx95fnr5e9l,{'claim': {'proof': ['a27e99ac4dfcdc581d9d9380...,{'claim': {'proof': ['a27e99ac4dfcdc581d9d9380...,claim,theprofit93,cosmos1734emz8rm2r54tf5gqj226dvv4phx95fssq2mc,91000000
13,bostrom197azlmrp8wvz4446jfnxu605snegznl479dz78,{'claim': {'proof': ['cad558f6f2b3a0bd3085c8a9...,{'claim': {'proof': ['cad558f6f2b3a0bd3085c8a9...,claim,mauro2019,cosmos197azlmrp8wvz4446jfnxu605snegznl4ake3qq,91000000
15,bostrom1nu0mxv9egu2tg7d93nqmnl85jc957ksg62fmew,{'claim': {'proof': ['19f71b0247bbdcb5dc3efc14...,{'claim': {'proof': ['19f71b0247bbdcb5dc3efc14...,claim,sanka76,osmo1nu0mxv9egu2tg7d93nqmnl85jc957ksg3zwc3m,91000000


In [6]:
gift_df = claim_txs_df.groupby('sender')['gift_claiming_address'].agg(pd.Series.nunique).reset_index().rename(
    columns={'gift_claiming_address': 'gift_claiming_addresses'})
gift_df.head()

,sender,gift_claiming_addresses
0,bostrom100c867fxeu5dq6ht0flc7vf0rl65sfm7ap0cgd,1
1,bostrom100ga50dzpxu0u5uen77alrjy2l6ctrnuewvjz5,1
2,bostrom100lngazcuyx9s7retesv34e447jadtg94wdt4g,2
3,bostrom100n73at5pewz0cmfzhptztlns6r96xgtu4zhw4,1
4,bostrom100qk2f338dh0r6g3tyw9tnvlywnjquyest0lka,1


### Create Sending Transactions

In [22]:
BOSTROM_CHAIN_ID = 'bostrom'
BOSTROM_NODE_LCD_URL = 'https://lcd.bostrom.bronbro.io/'
BOSTROM_LCD_CLIENT = LCDClient(chain_id=BOSTROM_CHAIN_ID, url=BOSTROM_NODE_LCD_URL)
START_SEQUENCE = 0
SENDINGS_PER_TX = 1_000
SENDER_ADDRESS = 'bostrom1zvyccfq73sj8cny3lk7d68d8mk5hvzx4r6nrg3'

for i in range(len(gift_df) // SENDINGS_PER_TX + 1):
    multi_send_output_list = []
    for _, gift_row in gift_df[i * SENDINGS_PER_TX:(i + 1) * SENDINGS_PER_TX].iterrows():
        multi_send_output_list.append(
            MultiSendOutput(
                gift_row.sender,
                Coins('{0}000millivolt,{0}000milliampere'.format(gift_row.gift_claiming_addresses))
            )
        )
    multi_send_input_list = [
        MultiSendInput(
            AccAddress(SENDER_ADDRESS),
            Coins('{0}000millivolt,{0}000milliampere'.format(
                sum(gift_df[i * SENDINGS_PER_TX:(i + 1) * SENDINGS_PER_TX].gift_claiming_addresses)))
        )
    ]
    tx = BOSTROM_LCD_CLIENT.tx.create(
        signers=[SignerOptions(address=AccAddress(SENDER_ADDRESS))],
        options=CreateTxOptions(
            msgs=[MsgMultiSend(multi_send_input_list, multi_send_output_list)],
            memo="https://cyb.ai/ipfs/QmUkpePsCX65RJzaZcX4fsdmMHfwKjcfHkv43dTcyWoS3Y",
            fee=Fee(24_000_000, Coins("240000boot"))
        ))
    with open(f'txs/{START_SEQUENCE + i}_unsigned_giftav_{i}.json', 'w') as tx_file:
        tx_file.write(json.dumps(tx.to_data()))